# Look at the file ripple-transactions-jan-2013-dec-2016

In [1]:
import pandas as pd

Let's import the data

In [2]:
data = pd.read_csv('../data/transactions/original/ripple-transactions-jan-2013-dec-2016.txt', header = None, names = ["hash", "sender", "receiver", "currency","amount1","amount2","ledger","sourceTag","DestinationTag","9","10"])

In [3]:
data.head()

,hash,sender,receiver,currency,amount1,amount2,ledger,sourceTag,DestinationTag,9,10
0,5C0504B229CD8EE803B01D78868B8F4B595D8B1D03636B...,rfe8yiZUymRPx35BEwGjhfkaLmgNsTytxT,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,XRP,0.0,100.0,7227428,NaN,NaN,1,1402920660
1,56022C64D29974C116591563DC8A80547907BE38EEB134...,rfe8yiZUymRPx35BEwGjhfkaLmgNsTytxT,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,XRP,0.0,100.0,7224947,NaN,NaN,1,1402908860
2,CCA2EB48BC106C7F4692BF71819EC236C9E09E37BCF38C...,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,rhjyoR8rpLMKM47pUgTjdNuVDJ1CrYYBwT,XRP,2000.0,0.0,5541569,NaN,NaN,1,1394924250
3,423708E30D94335FE2858C11B3D57390669A20FCE6C485...,rMTzGg7nPPEMJthjgEBfiPZGoAM7MEVa1r,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,XRP,2020.0,0.0,3333190,NaN,NaN,1,1384463290
4,C1E342D3168740197C7E6B84110CEF9A338EA0D0EF5697...,rLz6TEYTsSZivwgXssq8ZSmTsBSFTMCw7h,rffqWVgnv9bPKaWHHVUDMjdAEgab3LGjZ1,XRP,100.0,0.0,6965416,NaN,NaN,1,1401681860


By inspecting various fields and compring them in the data found on the https://xrpcharts.ripple.com/#/ it seems that
 - the first column is the hash
 - the second one is the sender address, 
 - third is receiver
 - fourth is currency
 - fifth and six are the amount of the transaction (since the data was in csv format and amount were given with a "," delimiter we need to combine fifth and 6 columns.
 - six is the ledger number
 - the columns seven and eight are respectively sourceTag and DestinationTag
 - 9 and 10 are not obvious at first look (And i don't know if it is a CSV error as in column 5 and 6)

Let's try to combine amount1 and amount2. The data in these two columns is in float format.<br>
We will first convert them to integers then to string and combine them with a "." and convert back to float.

In [4]:
print(data.amount1.isna().unique())
print(data.amount2.isna().unique())

[False  True]
[False  True]


It seems we have NA values in amount1 and amount2

In [5]:
data[data.amount1.isna()]

,hash,sender,receiver,currency,amount1,amount2,ledger,sourceTag,DestinationTag,9,10
24053310,E1709FAA24AEB0A9FE4B4471BF59A55052A33A1085604B...,NaN,NaN,NaN,NaN,NaN,23670686,NaN,NaN,2,1472360631


In [6]:
data[data.amount2.isna()]

,hash,sender,receiver,currency,amount1,amount2,ledger,sourceTag,DestinationTag,9,10
24053310,E1709FAA24AEB0A9FE4B4471BF59A55052A33A1085604B...,NaN,NaN,NaN,NaN,NaN,23670686,NaN,NaN,2,1472360631


Indeed at line 24053310 the data seems corrupted. Lets drop this line.

In [7]:
data.dropna(axis=0, how='any', subset= ["amount1","amount2"], inplace=True)

Now we can convert to int 

In [8]:
data.amount1 = data.amount1.astype('int64')
data.amount2 = data.amount2.astype('int64')

And combine amount1 and amount2 in a new column called amount.

In [9]:
data["amount"] = data.amount1.astype(str).str.cat(data.amount2.astype(str), sep='.')
data.amount = data.amount.astype('float64')
data.drop(["amount1", "amount2"], axis = 1) #remove old column
data = data[['hash','sender','receiver','currency','amount','ledger','sourceTag','DestinationTag','9','10']] #re-order columns

In [10]:
data

,hash,sender,receiver,currency,amount,ledger,sourceTag,DestinationTag,9,10
0,5C0504B229CD8EE803B01D78868B8F4B595D8B1D03636B...,rfe8yiZUymRPx35BEwGjhfkaLmgNsTytxT,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,XRP,1.000000e-01,7227428,NaN,NaN,1,1402920660
1,56022C64D29974C116591563DC8A80547907BE38EEB134...,rfe8yiZUymRPx35BEwGjhfkaLmgNsTytxT,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,XRP,1.000000e-01,7224947,NaN,NaN,1,1402908860
2,CCA2EB48BC106C7F4692BF71819EC236C9E09E37BCF38C...,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,rhjyoR8rpLMKM47pUgTjdNuVDJ1CrYYBwT,XRP,2.000000e+03,5541569,NaN,NaN,1,1394924250
3,423708E30D94335FE2858C11B3D57390669A20FCE6C485...,rMTzGg7nPPEMJthjgEBfiPZGoAM7MEVa1r,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,XRP,2.020000e+03,3333190,NaN,NaN,1,1384463290
4,C1E342D3168740197C7E6B84110CEF9A338EA0D0EF5697...,rLz6TEYTsSZivwgXssq8ZSmTsBSFTMCw7h,rffqWVgnv9bPKaWHHVUDMjdAEgab3LGjZ1,XRP,1.000000e+02,6965416,NaN,NaN,1,1401681860
5,F202202922C5D616096A51E76D0C65895620DEAB9517C3...,rGaRDdFcp1gZSrfuifZWiVLKq2SbaVRnuJ,rGaRDdFcp1gZSrfuifZWiVLKq2SbaVRnuJ,XRP,5.600000e-01,10438468,NaN,NaN,1,1418233430
6,5234065EB0115DFE0D1A71D7C2640EE798254A25E302D0...,rGaRDdFcp1gZSrfuifZWiVLKq2SbaVRnuJ,rfTMcYDpWXhgcpYK27qxzx4cT3qDEpzgfs,XRP,1.212782e+04,10438455,NaN,NaN,1,1418233370
7,358E5F076F3276C0516C271E1987BE619CB84ABF352B87...,rGaRDdFcp1gZSrfuifZWiVLKq2SbaVRnuJ,rGaRDdFcp1gZSrfuifZWiVLKq2SbaVRnuJ,XRP,8.700000e-01,10398519,NaN,NaN,1,1418061320
8,8ADB8D496F9F21799919F23FCF1B84A2A1EDCBA884B61A...,rGaRDdFcp1gZSrfuifZWiVLKq2SbaVRnuJ,rGaRDdFcp1gZSrfuifZWiVLKq2SbaVRnuJ,XRP,1.000000e+00,10398506,NaN,NaN,1,1418061260
9,40655777FE754D15872BB77943811C8596239BCBC21012...,rGaRDdFcp1gZSrfuifZWiVLKq2SbaVRnuJ,rGaRDdFcp1gZSrfuifZWiVLKq2SbaVRnuJ,XRP,1.212500e+04,10398452,NaN,NaN,1,1418061010


In [11]:
data.to_csv('../data/transactions/ripple-transactions-jan-2013-dec-2016.csv')